In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.datasets import load_diabetes, load_boston, load_breast_cancer
from typing import Dict, Tuple
from scipy import stats
from IPython.display import Image
from io import StringIO 
from IPython.display import Image
import graphviz 
import pydotplus
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score 
from sklearn.metrics import roc_curve, roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
from heamy.estimator import Regressor, Classifier
from heamy.pipeline import ModelsPipeline
from heamy.dataset import Dataset
from sklearn.linear_model import RidgeClassifier
%matplotlib inline 
sns.set(style="ticks")

In [2]:
breast_cancer = load_breast_cancer()
breast_cancer_X_train, breast_cancer_X_test, breast_cancer_y_train, breast_cancer_y_test = train_test_split(
    breast_cancer.data, breast_cancer.target, test_size=0.5, random_state=1)

In [3]:
# Обучим классификатор на 15 деревьях
bc1 = BaggingClassifier(n_estimators=15, oob_score=True, random_state=10)
bc1.fit(breast_cancer_X_train, breast_cancer_y_train)

BaggingClassifier(n_estimators=15, oob_score=True, random_state=10)

In [4]:
# Оценка качества модели
accuracy_score(breast_cancer_y_test, bc1.predict(breast_cancer_X_test))

0.9263157894736842

In [5]:
# Обучение модели бустинга
ab1 = AdaBoostClassifier(n_estimators=15, random_state=1)
ab1.fit(breast_cancer_X_train, breast_cancer_y_train)

AdaBoostClassifier(n_estimators=15, random_state=1)

In [6]:
# Оценка качества модели
accuracy_score(breast_cancer_y_test, ab1.predict(breast_cancer_X_test))

0.9228070175438596

In [7]:
dataset = Dataset(breast_cancer_X_train, breast_cancer_y_train, breast_cancer_X_test, breast_cancer_y_test)

model_tree = Regressor(dataset=dataset, estimator=DecisionTreeClassifier, name='tree')
model_lr = Regressor(dataset=dataset, estimator=RidgeClassifier,name='lr')
model_rf = Regressor(dataset=dataset, estimator=RandomForestClassifier, parameters={'n_estimators': 50},name='rf')

pipeline = ModelsPipeline(model_tree, model_lr, model_rf)
stack_ds = pipeline.stack(k=10, seed=1)
# модель второго уровня
stacker = Regressor(dataset=stack_ds, estimator=DecisionTreeClassifier)
results = stacker.validate(k=10,scorer=accuracy_score)

Metric: accuracy_score
Folds accuracy: [0.9655172413793104, 0.896551724137931, 0.9655172413793104, 0.9655172413793104, 0.9642857142857143, 0.9285714285714286, 0.9642857142857143, 1.0, 0.9642857142857143, 1.0]
Mean accuracy: 0.9614532019704433
Standard Deviation: 0.028876881324634177
Variance: 0.000833874275037006
